In [1]:
# import os
# # Create a folder for the pipeline step files
# experiment_folder = 'Ams_pipeline'
# os.makedirs(experiment_folder, exist_ok=True)

# print(experiment_folder)

In [2]:
# import azureml.core
# from azureml.core import Workspace

# # Load the workspace from the saved config file
# ws = Workspace.from_config()
# print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))
# ws.get_details()

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "tcp0044uc001-d993912"

try:
    # Check for existing compute target
    pipeline_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        pipeline_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        pipeline_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

NameError: name 'ws' is not defined

In [ ]:
%%writefile $experiment_folder/python_env2.yml

name: python_env2

dependencies:
- python
- anaconda
- pyarrow
- numpy
- pandas
- seaborn
- joblib
- shap
- graphviz
- tensorflow
- pip
- pip:
      - azureml.sdk
      - sklearn
      - statsmodels
     

In [ ]:

# from azureml.core import Environment
# from azureml.core.runconfig import RunConfiguration
# from azureml.core.runconfig import DockerConfiguration
# #from azureml.core import ScriptRunConfig

# # Create a Python environment for the experiment (from a .yml file)
# experiment_env = Environment.from_conda_specification("python_env2", experiment_folder + "/python_env2.yml")


# #Finally, use the environment in the ScriptRunConfig:
# # src = ScriptRunConfig(source_directory=experiment_folder,
#                        #script=experiment_env,
#                        #arguments=EXECUTE_ARGUMENTS,
#                        #compute_target=pipeline_cluster,
#                        #environment=experiment_env,
#                        #docker_runtime_config=docker_config)

# # Register the environment 
# experiment_env.register(workspace=ws)
# registered_env = Environment.get(ws, 'python_env2')

# # Create a new runconfig object for the pipeline
# pipeline_run_config = RunConfiguration()

# # Use the compute you created above. 
# pipeline_run_config.target = pipeline_cluster

# # Assign the environment to the run configuration
# pipeline_run_config.environment = registered_env

# print ("Run configuration created.")

In [ ]:
from azureml.data import OutputFileDatasetConfig
from azureml.pipeline.steps import PythonScriptStep

# Get the training dataset
# = ws.datasets.get("")

# Create an OutputFileDatasetConfig (temporary Data Reference) 
#prepped_data = OutputFileDatasetConfig("prepped_data")

# Run the data prep and train script
prep_train_step = PythonScriptStep(name = "Prepare & train Data",
                                source_directory = experiment_folder ,
                                script_name = "AMS_newdata_allmodels_ver2-copy.py",
                              #arguments = ['--input-data'.as_named_input('raw_data'),
                                             #'--prepped-data', prepped_data],
                                compute_target = pipeline_cluster,
                                runconfig = pipeline_run_config,
                                allow_reuse = True)

In [ ]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline


# Construct the pipeline
pipeline_steps = [prep_train_step]
pipeline = Pipeline(workspace=ws, steps=pipeline_steps)
print("Pipeline is built.")

# Create an experiment and run the pipeline
experiment = Experiment(workspace=ws, name = 'AMS_new_experiment')
pipeline_run = experiment.submit(pipeline, regenerate_outputs=True)
print("Pipeline submitted for execution.")

pipeline_run.wait_for_completion(show_output=True)